In [65]:
import yfinance as yf
import math

In [72]:
stock = yf.Ticker('TSLA')
stockInfo = stock.info

2.92


# EPS

In [67]:
def calc_eps(net_income_to_common, shares_outstanding):
    return net_income_to_common / shares_outstanding

calc_eps(stockInfo['netIncomeToCommon'], stockInfo['sharesOutstanding'])

3.038484681151332

# Volatility
Valid intervals: `[1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]`

In [ ]:
def calc_volatility(stock, period, interval):
    historic = stock.history(period=period, interval=interval)
    values = [x for x in historic['Close'] if not math.isnan(x)]
    print(historic['Close'], len(historic['Close']))

    mean = sum(values) / len(values)
    deviation = [price - mean for price in values]
    variance = sum([dev * dev for dev in deviation]) / len(values)
    std_dev = math.sqrt(variance)

    #Not completely sure if this is the exact formula that is needed/wanted?
    volatility = std_dev * math.sqrt(len(values))
    
    return volatility
    


calc_volatility(stock, '2mo', '1d')


# Estimate future price

In [82]:
gr_values = {
    'Total Assets': 0.1,
    'Total Current Assets': 0.05,
    'Retained Earnings': 0.05,
    'Gross Profit': 0.2,
    'Net Income': 0.2,
    'Total Cash From Operating Activities': 0.15
}

def calc_growth_rate(stock, eps_an, years):
    result = 0

    for i, (data, weight) in enumerate(gr_values.items()):
        if i < 3:
            source = stock.balancesheet
        elif i < 5:
            source = stock.financials
        else:
            source = stock.cashflow
        
        data_gr = 0
        for j, year in enumerate(years):
            if j == len(years) - 1:
                break
                
            present = source[year][data]
            past = source[years[j + 1]][data]
            data_gr += ((present - past) / past)
        

        result += ((data_gr / 2) * weight)
    
    #Specific for EPS
    eps_gr = 0
    for i, eps in enumerate(eps_an):
        if i == len(years) - 1:
            break
                
        past = eps_an[i + 1]
        eps_gr += ((eps - past) / past)
    result += ((eps_gr / 2) * 0.25)


    return result


#returns stockprice in 3 years
def get_future_price(stock):
    historic = stock.history(start='2019-01-01', end='2021-12-31', interval='1mo')['Close']
    fc = historic[historic.notnull()]
    financial_years = stock.financials.keys()[:3]

    eps_an = [round(stock.financials[year]['Net Income Applicable To Common Shares'] / stock.shares['BasicShares'][year.year], 2) for year in financial_years]

    #Calculate future p/e ratio
    closes_an = [fc[ts] for ts in fc.keys() if ts.month == 12]
    closes_an.reverse()

    tot_pe = [price / eps_an[i] for i, price in enumerate(closes_an)]
    pe = round(sum(tot_pe) / len(tot_pe), 2)

    #Calculate future eps
    eps = eps_an[0] * ((1 + calc_growth_rate(stock, eps_an, financial_years)) ** len(financial_years))

    return pe * eps
    

print(f"Future (3y) price of {stock.info['symbol']}: {get_future_price(stock)}")

5524.0
